# Keras's Finetune

keras 实现线性模型

In [1]:
from utils import *
from keras.optimizers import SGD, RMSprop, Adam

Using TensorFlow backend.


In [2]:
x = random((30, 2))
x[:3]

array([[ 0.4272,  0.6899],
       [ 0.9954,  0.164 ],
       [ 0.1836,  0.522 ]])

In [3]:
y = x.dot([2., 3.]) + 1.
y[:3]

array([ 3.9242,  3.4829,  2.9333])

In [4]:
lm = Sequential([Dense(1, input_shape=(2,))])
lm.compile(optimizer=SGD(lr=.1), loss='mse')

In [5]:
lm.fit(x, y, nb_epoch=10, batch_size=1)

Epoch 1/10
30/30 [==============================] - 0s - loss: 1.8970       
Epoch 2/10
30/30 [==============================] - 0s - loss: 0.1802     
Epoch 3/10
30/30 [==============================] - 0s - loss: 0.0791     
Epoch 4/10
30/30 [==============================] - 0s - loss: 0.0385     
Epoch 5/10
30/30 [==============================] - 0s - loss: 0.0206     
Epoch 6/10
30/30 [==============================] - 0s - loss: 0.0108     
Epoch 7/10
30/30 [==============================] - 0s - loss: 0.0070     
Epoch 8/10
30/30 [==============================] - 0s - loss: 0.0033         
Epoch 9/10
30/30 [==============================] - 0s - loss: 0.0019     
Epoch 10/10
30/30 [==============================] - 0s - loss: 8.4870e-04 


In [6]:
lm.get_weights()

[array([[ 1.9327],
        [ 2.9479]], dtype=float32), array([ 1.0599], dtype=float32)]

 VGG change

In [2]:
import vgg16
from vgg16 import Vgg16

In [3]:
vgg = Vgg16()

In [9]:
# path = 'data/redux/'
path = 'data/redux/sample/'
model_path = path + 'models/'

In [10]:
BATCH_SIZE = 8

In [11]:
val_batches = get_batches(path + 'valid', shuffle=False, batch_size=BATCH_SIZE)
batches = get_batches(path + 'train', shuffle=False, batch_size=BATCH_SIZE)

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [12]:
#batches.nb_sample
#batches.next()

In [13]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [14]:
# val_batches

In [15]:
# val_data = get_data(path + 'valid')

In [16]:
# trn_data = get_data(path + 'train')

In [17]:
# save_array(model_path + 'train_data.bc', trn_data)
# save_array(model_path + 'valid_data.bc', val_data)

In [18]:
# trn_data = load_array(model_path + 'train_data.bc')
# val_data = load_array(model_path + 'valid_data.bc')

In [19]:
# trn_data.shape

In [20]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [21]:
trn_labels.shape

(200, 2)

In [22]:
# trn_features = model.predict(trn_data, batch_size=BATCH_SIZE)
# trn_features

In [23]:
# val_features = model.predict(val_data, batch_size=BATCH_SIZE)
# val_features

In [25]:
val_features = vgg.model.predict_generator(val_batches, val_batches.nb_sample)

In [26]:
val_features.shape

(50, 1000)

In [29]:
trn_features = vgg.model.predict_generator(batches, batches.nb_sample)

In [31]:
trn_features.shape

(200, 1000)

In [32]:
# vgg.compile()
# vgg.fit(batches, val_batches, nb_epoch=1)

In [33]:
save_array(model_path + 'train_lastlayer_features.bc', trn_features)
save_array(model_path + 'valid_lastlayer_features.bc', val_features)

In [34]:
trn_features = load_array(model_path + 'train_lastlayer_features.bc')
val_features = load_array(model_path + 'valid_lastlayer_features.bc')

In [35]:
lm = Sequential([Dense(2, activation='softmax', input_shape=(1000, ))])
lm.compile(optimizer=RMSprop(lr=.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=BATCH_SIZE, validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/3
200/200 [==============================] - 0s - loss: 0.5149 - acc: 0.7050 - val_loss: 0.3934 - val_acc: 0.8200
Epoch 2/3
200/200 [==============================] - 0s - loss: 0.3255 - acc: 0.8800 - val_loss: 0.3709 - val_acc: 0.8000
Epoch 3/3
200/200 [==============================] - 0s - loss: 0.2528 - acc: 0.9200 - val_loss: 0.3846 - val_acc: 0.8200


In [37]:
lm.fit(trn_features, trn_labels, nb_epoch=6, batch_size=BATCH_SIZE, validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/6
200/200 [==============================] - 0s - loss: 0.2051 - acc: 0.9300 - val_loss: 0.4178 - val_acc: 0.8200
Epoch 2/6
200/200 [==============================] - 0s - loss: 0.1762 - acc: 0.9250 - val_loss: 0.4092 - val_acc: 0.8400
Epoch 3/6
200/200 [==============================] - 0s - loss: 0.1455 - acc: 0.9550 - val_loss: 0.4064 - val_acc: 0.8000
Epoch 4/6
200/200 [==============================] - 0s - loss: 0.1291 - acc: 0.9550 - val_loss: 0.4262 - val_acc: 0.8200
Epoch 5/6
200/200 [==============================] - 0s - loss: 0.1105 - acc: 0.9550 - val_loss: 0.4369 - val_acc: 0.8000
Epoch 6/6
200/200 [==============================] - 0s - loss: 0.0983 - acc: 0.9700 - val_loss: 0.4551 - val_acc: 0.8000


In [38]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_4 (Dense)                  (None, 2)             2002        dense_input_1[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________


In [41]:
vgg.model.pop()

In [43]:
for layer in vgg.model.layers: layer.trainable=False

In [46]:
vgg.model.add(Dense(2, activation='softmax'))

In [48]:
# gen=image.ImageDataGenerator()
# batches = gen.flow(trn_data, trn_labels, batch_size=BATCH_SIZE, shuffle=True)
# val_batches = gen.flow(val_data, val_labels,)

In [49]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch, validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [51]:
opt = RMSprop(lr=.1)
vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
fit_model(vgg.model, batches, val_batches, nb_epoch=3)

Epoch 1/3
200/200 [==============================] - 11s - loss: 8.1396 - acc: 0.4950 - val_loss: 7.0920 - val_acc: 0.5600
Epoch 2/3
200/200 [==============================] - 11s - loss: 8.1396 - acc: 0.4950 - val_loss: 7.0920 - val_acc: 0.5600
Epoch 3/3
200/200 [==============================] - 11s - loss: 8.1396 - acc: 0.4950 - val_loss: 7.0920 - val_acc: 0.5600
